<a href="https://colab.research.google.com/github/MariiaSam/DS_homeWork/blob/main/HW_6_corr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [147]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [148]:
file_path = '/content/drive/My Drive/Housing.csv'
df = pd.read_csv(file_path)
df

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [149]:
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [150]:
df.isnull().sum()

,0
price,0
area,0
bedrooms,0
bathrooms,0
stories,0
mainroad,0
guestroom,0
basement,0
hotwaterheating,0
airconditioning,0


In [151]:
df.describe()

,price,area,bedrooms,bathrooms,stories,parking
count,5.450000e+02,545.000000,545.000000,545.000000,545.000000,545.000000
mean,4.766729e+06,5150.541284,2.965138,1.286239,1.805505,0.693578
std,1.870440e+06,2170.141023,0.738064,0.502470,0.867492,0.861586
min,1.750000e+06,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,3.430000e+06,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,4.340000e+06,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,5.740000e+06,6360.000000,3.000000,2.000000,2.000000,1.000000
max,1.330000e+07,16200.000000,6.000000,4.000000,4.000000,3.000000


In [152]:
# Гіпотеза (функція передбачення)
def hypothesis(X, w):
    return np.dot(X, w)

In [153]:
# Функція для обчислення втрат (MSE)
def compute_cost(X, y, w):
    m = len(y)
    error = hypothesis(X, w) - y
    return np.sum(error ** 2) / (2 * m)

In [154]:
# Один крок градієнтного спуску
def gradient_step(X, y, w, alpha=0.01):
    m = len(y)
    error = hypothesis(X, w) - y
    gradient = (1 / m) * np.dot(X.T, error)
    w = w - alpha * gradient
    return w


In [155]:
# Градієнтний спуск
def gradient_descent(X, y, w, alpha, num_iters):
    cost_history = []

    for i in range(num_iters):
        w = gradient_step(X, y, w, alpha)
        cost = compute_cost(X, y, w)
        cost_history.append(cost)

    return w, cost_history

In [156]:
# Аналітичне рішення
def analytical_solution(X, y):
    X_transpose = np.transpose(X)
    w = np.linalg.inv(X_transpose.dot(X)).dot(X_transpose).dot(y)
    return w

In [157]:
# Обчислення втрат (середньоквадратичної помилки (MSE))
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

In [158]:
# Основний код
X = df[['area', 'bathrooms', 'bedrooms']].values
y = df['price'].values


In [159]:
# Масштабування даних
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [160]:
# Додаємо стовпець з 1 для вільного члена (інтерсепт)
X_scaled = np.c_[np.ones(X_scaled.shape[0]), X_scaled]

In [161]:
# 1. Градієнтний спуск
initial_w = np.zeros(X_scaled.shape[1])
alpha = 0.01  # Підвищуємо коефіцієнт навчання після масштабування
num_iters = 10000
w_gd, cost_history = gradient_descent(X_scaled, y, initial_w, alpha, num_iters)
print(f'Ваги з градієнтного спуску (масштабовані): {w_gd}')

# 2. Аналітичне рішення
w_analytical = analytical_solution(X_scaled, y)
print(f'Ваги з аналітичного рішення (масштабовані): {w_analytical}')

Ваги з градієнтного спуску (масштабовані): [4766729.24770638  821214.14349519  695808.52272536  299983.57107964]
Ваги з аналітичного рішення (масштабовані): [4766729.24770642  821214.14349519  695808.52272538  299983.57107963]


In [162]:
# 3. Scikit-learn
model = LinearRegression()
model.fit(X_scaled[:, 1:], y)  # Пропускаємо перший стовпець (інтерсепт)
w_sklearn = np.concatenate([[model.intercept_], model.coef_])
print(f'Ваги з scikit-learn (масштабовані): {w_sklearn}')

Ваги з scikit-learn (масштабовані): [4766729.24770642  821214.14349519  695808.52272537  299983.57107963]


In [163]:
# Прогнози для кожного методу
y_pred_gd = hypothesis(X_scaled, w_gd)  # Градієнтний спуск
y_pred_analytical = hypothesis(X_scaled, w_analytical)  # Аналітичне рішення
y_pred_sklearn = hypothesis(X_scaled, w_sklearn)  # Scikit-learn

In [164]:
# Виводимо MSE для кожного методу
mse_gd = mse(y, y_pred_gd)
mse_analytical = mse(y, y_pred_analytical)
mse_sklearn = mse(y, y_pred_sklearn)

print(f'MSE з градієнтного спуску: {mse_gd}')
print(f'MSE з аналітичного рішення: {mse_analytical}')
print(f'MSE з scikit-learn: {mse_sklearn}')


MSE з градієнтного спуску: 1791170049977.3193
MSE з аналітичного рішення: 1791170049977.3193
MSE з scikit-learn: 1791170049977.3193


Ваги з градієнтного спуску (масштабовані):
[4766729.24770638, 821214.14349519, 695808.52272536, 299983.57107964]

Ваги з аналітичного рішення (масштабовані):
[4766729.24770642, 821214.14349519, 695808.52272538, 299983.57107963]

Ваги з Scikit-learn (масштабовані):
[4766729.24770642, 821214.14349519, 695808.52272537, 299983.57107963]

Значення середньоквадратичної помилки (MSE):
Градієнтний спуск: 1791170049977.3193

Аналітичне рішення: 1791170049977.3193

Scikit-learn: 1791170049977.3193

**Висновок:
Оскільки всі методи дали однакові або дуже близькі значення ваг і MSE, можна стверджувати, що вони працюють коректно і дають аналогічні результати. Це підтверджує, що алгоритми градієнтного спуску, аналітичного рішення та Scikit-learn коректно працюють для цієї задачі регресії.**